In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime
from Create_Table_queries import *


def process_youtubedata_file(cur, filepath): 
    """
        This function reads one JSON file and read information of videos and youtuber data and saves into video_data and youtuber_data
        Arguments:
        cur: Database Cursor
        filepath: location of JSON files
        Return: None
    """
    # open JSON file
    df = pd.read_json(filepath, lines=True)
    
    # ---------insert youtuber record----------
    # write your code here that reads youtuber data from JSON file and insert it into Youtubers_dim table 

    # write your code here
    for indx, row in df.iterrows():
        youtuber_record = [ row['youtuber_id']['0'],row['youtuber_name']['0'],row['youtuber_location']['0'],row['youtuber_latitude']['0'],row['youtuber_longitude']['0'] ]
        #print(youtuber_record)
        cur.execute("Select count(*) from youtubers_dim where youtuber_Id='"+str(row['youtuber_id']['0'])+"';")
        res  = cur.fetchone()
        if res[0] == 0:
            cur.execute(Youtubers_table_insert, youtuber_record)


    # ---------insert video record--------------
    # write your code here that reads youtube videos data from JSON file and insert it into Videos_dim table 

    # write your code here
    for indx, row in df.iterrows():
        video_record = [ row['video_id']['0'],row['title']['0'],row['youtuber_id']['0'],row['year']['0'],row['duration']['0'] ]
        #print(video_record)
        cur.execute("Select count(*) from videos_dim where video_Id='"+str(row['video_id']['0'])+"';")
        res  = cur.fetchone()
        if res[0] == 0:
            cur.execute(Videos_table_insert, video_record)
            


def process_log_file(cur, filepath):
    """
        This function reads Log files and reads information of time, user and videoplay data and saves into time, user, videoplay
        Arguments:
        cur: Database Cursor
        filepath: location of Log files
        Return: None
        
    """

    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextVideo action
    df = df[(df['page'] == 'NextVideo')]

    # convert timestamp column to datetime    
    # insert time data records to Time_dim table
        # write your code here

    for ind, row in df.iterrows():
        date = pd.to_datetime(row['ts'], unit='ms').date()
        hr = pd.to_datetime(row['ts'], unit='ms').hour
        day = pd.to_datetime(row['ts'], unit='ms').dayofyear
        week = pd.to_datetime(row['ts'], unit='ms').week
        month = pd.to_datetime(row['ts'], unit='ms').month
        year = pd.to_datetime(row['ts'], unit='ms').year
        week_day = pd.to_datetime(row['ts'], unit='ms').dayofweek
        

        #print(row)
        cur.execute("SELECT COUNT(*) FROM TIME_DIM WHERE start_time='"+str(date)+"';")
        res = cur.fetchone()
        if res[0] == 0:
            time_record = [date, hr, day, week, month, year, week_day]
            cur.execute(Time_table_insert, time_record)
            
            
    # load user table
    
    # insert user records into Users_dim table
        # write your code here
    for ind, row in df.iterrows():
        user_record = [ row['userId'], row['firstName'], row['lastName'], row['gender'], row['level']  ]
        #print(user_record)
        cur.execute("Select count(*) from users_dim where user_Id="+str(row['userId'])+";")
        res  = cur.fetchone()
        if res[0] == 0:
            cur.execute(Users_table_insert, user_record)

    # insert Videoplay records in Videoplay_fact table
        # write your code here

    for ind, row in df.iterrows():
        vid_name = list(row['video'])
        vid_name = "".join(["''" if i=="'" else i for i in vid_name])

        cur.execute("SELECT video_id, youtuber_id from videos_dim WHERE title LIKE '%"+vid_name+"';")
        video_Id = None
        youtuber_Id = None
        result_row = cur.fetchone() 
        if result_row:
            video_Id = result_row[0]
            youtuber_Id = result_row[1]

        vp_record = [ind, pd.to_datetime(row['ts'], unit='ms').date(), row['userId'], row['level'], video_Id , youtuber_Id, row['sessionId'], row['location'] ,row['userAgent']]
    
        cur.execute(Videoplay_table_insert, vp_record)
        

def process_data(cur, conn, filepath, func):
    """
        This function get all JSON files in given directory by exploring all sub directories, an++d process all files that were found using the given function.
        Example: if I give it the path to youtube_data directory which resides in data folder of this assignment,
        and func given is process_youtubedata_file it should get all JSON files in this directories and process each file using process_youtubedata_file function. 
        Arguments:
        cur: Database Cursor
        conn: Database
        filepath: location of JSON files
        func: function to process all files in the directory
        Return: None
    """
    for sd,dr, files in os.walk(filepath):
        for  file in files:
            if file.endswith(".json"):
                func(cur, os.path.join(sd,file) )



def main():
    conn = psycopg2.connect("host=127.0.0.1 dbname=youtubedb user=postgres password=12345678")
    cur = conn.cursor()
    conn.set_session(autocommit=True)
    
    print("youtube data")
    process_data(cur, conn, filepath='data/youtube_data', func=process_youtubedata_file)
    print("----"*3,"\nLog data\n")
    process_data(cur, conn, filepath='data/log_data', func=process_log_file)

    conn.close()
    print("\nconnection closed!")
    
    
if __name__ == "__main__":
    main()

youtube data
------------ 
Log data


connection closed!
